Note: There might be a handful of bugs at the moment. The developers of this stable diffusion implementation keep changing the api. Everyone should know not to make breaking api changes so regularly! I'll do a pass over the code and fix bugs as soon as I can. Am away this week :) thanks to Michael d for bringing this to my attention. 

# Stable Diffusion Initialisation

In [1]:
# !pip install transformers diffusers lpips 

In [2]:
with open("scratch/token.txt") as f:
    token = f.read().strip()

In [3]:
from typing import Any, List
from pathlib import Path
from pydantic import BaseModel
import json

import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
from matplotlib import pyplot as plt
import numpy
from torchvision import transforms as tfms
import numpy as np

# For video display:
from IPython.display import HTML
from base64 import b64encode

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

print(torch_device)

cuda


/home/jerome/miniconda3/envs/ldm/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [4]:
# Using torchvision.transforms.ToTensor
to_tensor_tfm = tfms.ToTensor()

def pil_to_latent(input_im):
  # Single image -> single latent in a batch (so size 1, 4, 64, 64)
  with torch.no_grad():
    latent = vae.encode(to_tensor_tfm(input_im).unsqueeze(0).to(torch_device)*2-1) # Note scaling
  return 0.18215 * latent.mode() # or .mean or .sample

def latents_to_pil(latents):
    
  # batch of latents -> list of images
  latents = (1 / 0.18215) * latents
    
  with torch.no_grad():
    image = vae.decode(latents)[0]
    
  image = (image / 2 + 0.5).clamp(0, 1)
  image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
  images = (image * 255).round().astype("uint8")
  pil_images = [Image.fromarray(image) for image in images]
  return pil_images

# potentially list of str needed?

def prep_text(
    prompt: str,
    tokenizer: Any,
    max_length: int = None
):
    max_length = max_length or tokenizer.model_max_length
    
    if isinstance(prompt, str):
        prompt = [prompt]
    
    # Prep text 
    text_input = tokenizer(
        prompt, 
        padding="max_length", 
        max_length=tokenizer.model_max_length, 
        truncation=True, 
        return_tensors="pt",
    )

    with torch.no_grad():
      text_embeddings = text_encoder(
          text_input.input_ids.to(torch_device)
      )[0]

    max_length = text_input.input_ids.shape[-1]
    return text_embeddings, max_length


def generate_image(
    text_embeddings, # tensor?
    loading_bar=True,
    batch_size=1,
    generator=None,
    guidance_scale=7.5,
    num_inference_steps=50,    
    height=512,
    width=768,
    latents=None
    
):
    if generator is None:
        generator = torch.manual_seed(42)
    
    # Prep Scheduler
    scheduler.set_timesteps(num_inference_steps)
        
    # Prep latents
    if latents is None:
        latents = torch.randn(
            (
                batch_size, 
                unet.in_channels, 
                height // 8, 
                width // 8
            ),
            generator=generator,
        )

    latents = latents.to(torch_device)
    latents = latents * scheduler.sigmas[0] # Need to scale to match k
    
    loading_bar_function = tqdm if loading_bar else lambda x: x

    # Loop
    with autocast("cuda"):
        for i, t in loading_bar_function(enumerate(scheduler.timesteps)):

            # expand the latents if we are doing classifier-free guidance 
            # to avoid doing two forward passes.
            latent_model_input = torch.cat([latents] * 2)
            sigma = scheduler.sigmas[i]
            latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

            # predict the noise residual
            with torch.no_grad():
                noise_pred = unet(
                    latent_model_input, 
                    t, 
                    encoder_hidden_states=text_embeddings,
                )["sample"]

            # perform guidance
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

            # compute the previous noisy sample x_t -> x_t-1
            latents = scheduler.step(noise_pred, i, latents)["prev_sample"]

    return latents_to_pil(latents)[0]

def get_output_dir(base: Path):
    base.mkdir(exist_ok=True, parents=True)
    index = max(map(lambda p: int(p.stem), base.glob("[0-9]*")), default=0) + 1
    interp_output = base / str(index)
    interp_output.mkdir()
    return interp_output

In [5]:
# Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("stabilityai/stable-diffusion-2", subfolder="vae", use_auth_token=token)

In [6]:
# Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.15.self_attn.k_proj.bias', 'vision_model.encoder.layers.3.mlp.fc2.weight', 'vision_model.encoder.layers.15.layer_norm1.bias', 'vision_model.encoder.layers.2.layer_norm2.bias', 'vision_model.encoder.layers.14.mlp.fc2.weight', 'vision_model.encoder.layers.2.mlp.fc1.weight', 'vision_model.encoder.layers.11.layer_norm1.bias', 'vision_model.encoder.layers.7.mlp.fc1.bias', 'vision_model.encoder.layers.1.self_attn.q_proj.weight', 'vision_model.encoder.layers.12.layer_norm2.weight', 'vision_model.encoder.layers.23.mlp.fc1.bias', 'vision_model.encoder.layers.17.self_attn.out_proj.bias', 'vision_model.encoder.layers.0.self_attn.q_proj.weight', 'vision_model.encoder.layers.21.layer_norm1.weight', 'vision_model.encoder.layers.14.self_attn.v_proj.weight', 'vision_model.encoder.layers.2.mlp.fc2.bias', 'vision_model.encoder.layers.6.self_attn.out_proj.weig

In [7]:
# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("stabilityai/stable-diffusion-2", subfolder="unet", use_auth_token=token)

The config attributes {'dual_cross_attention': False, 'use_linear_projection': True} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ <ipython-input-7-c5d8707a32b7>:2 in <module>                                                     │
│                                                                                                  │
│ /home/jerome/miniconda3/envs/ldm/lib/python3.8/site-packages/diffusers/modeling_utils.py:451 in  │
│ from_pretrained                                                                                  │
│                                                                                                  │
│   448 │   │   if low_cpu_mem_usage:                                                              │
│   449 │   │   │   # Instantiate model with empty weights                                         │
│   450 │   │   │   with accelerate.init_empty_weights():                                          │
│ ❱ 451 │   │   │   │   model, unused_kwargs = cls.from_config(                                    │
│   452 │   │   │   │   │   config_path,                                                           │
│   453 │   │   │   │   │   cache_dir=cache_dir,                                                   │
│   454 │   │   │   │   │   return_unused_kwargs=True,                                             │
│                                                                                                  │
│ /home/jerome/miniconda3/envs/ldm/lib/python3.8/site-packages/diffusers/configuration_utils.py:17 │
│ 4 in from_config                                                                                 │
│                                                                                                  │
│   171 │   │   │   init_dict["dtype"] = unused_kwargs.pop("dtype")                                │
│   172 │   │                                                                                      │
│   173 │   │   # Return model and optionally state and/or unused_kwargs                           │
│ ❱ 174 │   │   model = cls(**init_dict)                                                           │
│   175 │   │   return_tuple = (model,)                                                            │
│   176 │   │                                                                                      │
│   177 │   │   # Flax schedulers have a state, so return it.                                      │
│                                                                                                  │
│ /home/jerome/miniconda3/envs/ldm/lib/python3.8/site-packages/diffusers/configuration_utils.py:44 │
│ 4 in inner_init                                                                                  │
│                                                                                                  │
│   441 │   def inner_init(self, *args, **kwargs):                                                 │
│   442 │   │   # Ignore private kwargs in the init.                                               │
│   443 │   │   init_kwargs = {k: v for k, v in kwargs.items() if not k.startswith("_")}           │
│ ❱ 444 │   │   init(self, *args, **init_kwargs)                                                   │
│   445 │   │   if not isinstance(self, ConfigMixin):                                              │
│   446 │   │   │   raise RuntimeError(                                                            │
│   447 │   │   │   │   f"`@register_for_config` was applied to {self.__class__.__name__} init m   │
│                                                                                                  │
│ /home/jerome/miniconda3/envs/ldm/lib/python3.8/site-packages/diffusers/models/unet_2d_condition. │
│ py:135 in __init__                                                                               │
│                                                                                                  │
│   132 │   │   │   output_channel = block_out_channels[i]   

In [ ]:

# The noise scheduler
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

# Inference

In [ ]:
output_dir = "outputs/"

In [ ]:
prompt = [
    "Fjords"
]
height = 512
width = 768
num_inference_steps = 50
guidance_scale = 7.5
batch_size = 1


text_embeddings, max_length = prep_text(prompt, tokenizer)
uncond_embeddings, _ = prep_text([""], tokenizer)


text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

for i in range(3):
    display(generate_image(
        text_embeddings,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=torch.manual_seed(i+1),
        batch_size=batch_size,
    ))

# Concept interpolation

## Two step

In [ ]:
current_output_dir = get_output_dir(Path(output_dir) / "interp")
# current_output_dir = Path(output_dir) / "interp" / "4"


prompt_1 = [
    "ancient village with small huts, two men with spears talking"
]
prompt_2 = [
    "futuristic city, flying cars, city lights, busy street, 4k, photograph of the year"
]
height = 512
width = 768
num_inference_steps = 50
guidance_scale = 7.5
generator = torch.manual_seed(42)
batch_size = 1

images = []


text_embeddings_1, max_length = prep_text(prompt_1, tokenizer)
text_embeddings_2, max_length = prep_text(prompt_2, tokenizer)

delta = text_embeddings_2 - text_embeddings_1

for i, fraction in tqdm(enumerate(np.linspace(0, 1))):

    text_embeddings_combined = text_embeddings_1 + delta*fraction

    uncond_embeddings, _ = prep_text([""], tokenizer)
    text_embeddings = torch.cat([uncond_embeddings, text_embeddings_combined])

    im = generate_image(
        text_embeddings,
        loading_bar=False,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=torch.manual_seed(42),
        batch_size=batch_size,
    )
    images.append(im)
    im.save(current_output_dir / f"image_{i}.png")

## N step

In [ ]:
class NStepConfig(BaseModel):
    prompts: List[str]
    n_interp_steps: int = 60
        
class _Step(BaseModel):
    origin_image_prompt: str
    origin_image_index: int
    target_image_prompt: str
    target_image_index: int
    file_name: str
    fraction: float
        
class NStepSnapshotConfig(BaseModel):
    config: NStepConfig
    steps: List[_Step]

In [ ]:
current_output_dir = get_output_dir(Path(output_dir) / "interp_n_steps")

factor = 1
height = int(512*factor)
width = int(768*factor)
num_inference_steps = 50
guidance_scale = 7.5
get_generator = lambda: torch.manual_seed(43)
batch_size = 1

config = NStepConfig(
    n_interp_steps=60,
    prompts=[
        "Vladimir Putin",
        "Donald Trump",
        "A baby in a suit",
        "Vladimir Putin",
    ],
)

embeddings = [
    prep_text([prompt], tokenizer)[0]
    for prompt in config.prompts
]

deltas = [
    embeddings[i+1] - embeddings[i]
    for i in range(len(embeddings) - 1)
]

# scout
print("scouting")

for i, e in enumerate(embeddings):
    (current_output_dir / "scout").mkdir(exist_ok=True)
    
    uncond_embeddings, _ = prep_text([""], tokenizer)
    e = torch.cat([uncond_embeddings, e])
    
    generate_image(
        e,
        loading_bar=True,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=get_generator(),
        batch_size=batch_size,
    ).save(current_output_dir / "scout" / f"{i}.png")

print("scouting - complete")

tmp = [
    (i, frac )
    for i in range(len(config.prompts) - 1)
    for frac in np.linspace(0, 1, config.n_interp_steps)
]

steps = [
    _Step(
        origin_image_index=i,
        origin_image_prompt=config.prompts[i],
        target_image_index=i+1,
        target_image_prompt=config.prompts[i+1],
        fraction=frac,
        file_name=f"image_{c:05d}_{i+frac:.5}.png",
    )
    for c, (i, frac) in enumerate(tmp)
]

snapshot_config = NStepSnapshotConfig(config=config, steps=steps)

with open(current_output_dir / "config.json", "w") as f:
    f.write(snapshot_config.json(indent=4))

    
for step in tqdm(steps):
    fn = current_output_dir / step.file_name
    if fn.exists():
        continue
    
    text_embeddings_1 = embeddings[step.origin_image_index]
    delta = deltas[step.origin_image_index]
    text_embeddings_combined = text_embeddings_1 + delta*step.fraction

    uncond_embeddings, _ = prep_text([""], tokenizer)
    text_embeddings = torch.cat([uncond_embeddings, text_embeddings_combined])

    im = generate_image(
        text_embeddings,
        loading_bar=False,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=get_generator(),
        batch_size=batch_size,
    )
    im.save(current_output_dir / step.file_name)

# Variable embedding step

In [ ]:
current_output_dir = get_output_dir(Path(output_dir) / "interp_variable_step")


prompt_1 = [
    "ancient village with small huts, two men with spears talking"
]
prompt_2 = [
    "futuristic city, flying cars, city lights, busy street, 4k, photograph of the year"
]
height = 512 // 2
width = 768 // 2
num_inference_steps = 50
guidance_scale = 7.5
generator = torch.manual_seed(42)
batch_size = 1

images = []


text_embeddings_1, max_length = prep_text(prompt_1, tokenizer)
text_embeddings_2, max_length = prep_text(prompt_2, tokenizer)

delta = text_embeddings_2 - text_embeddings_1

for i, fraction in tqdm(enumerate(np.linspace(0, 1))):

    text_embeddings_combined = text_embeddings_1 + delta*fraction

    uncond_embeddings, _ = prep_text([""], tokenizer)
    text_embeddings = torch.cat([uncond_embeddings, text_embeddings_combined])

    current_im = generate_image(
        text_embeddings,
        loading_bar=False,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=torch.manual_seed(42),
        batch_size=batch_size,
    )
    current_im.save(interp_output / f"image_{i}.png")
    
    if i>2:
        break
    
    prev_im = current_im

In [ ]:
current_im

In [ ]:
prev_im

In [ ]:
from PIL import Image, ImageChops

diff = ImageChops.difference(prev_im, current_im)
diff

In [ ]:
data = diff.getdata()
data.

# Noise Latent Pertubation

In [ ]:
current_output_dir = get_output_dir(Path(output_dir) / "interp_latent")
# current_output_dir = Path(output_dir) / "interp_latent" / "1"


prompt = [
    "A litter of shih tzu puppies"
]
height = 512
width = 768
num_inference_steps = 50
guidance_scale = 7.5
generator = torch.manual_seed(42)
batch_size = 1


n_points_interp = 180

latents_list = [
    torch.randn(
        (
            batch_size, 
            unet.in_channels, 
            height // 8, 
            width // 8
        ),
        generator=torch.manual_seed(i),
    )
    for i in [42, 2, 3, 4]
]
latents_list += [latents_list[0]]

steps = [
    (i, frac)
    for i in range(len(latents_list) - 1)
    for frac in np.linspace(0, 1, n_points_interp)
]

text_embeddings_orig, max_length = prep_text(prompt, tokenizer)
uncond_embeddings, _ = prep_text([""], tokenizer)
text_embeddings = torch.cat([uncond_embeddings, text_embeddings_orig])


for c, (i, fraction) in tqdm(list(enumerate(steps))):
    im_file = current_output_dir / f"image_{c:05d}.png"
    if im_file.exists():
        continue
    
    latents_1 = latents_list[i]
    latents_2 = latents_list[i+1]
    
    delta = latents_2 - latents_1
    latents = latents_1 + delta*fraction
    latents -= latents.mean()
    latents /= latents.std()
    im = generate_image(
        text_embeddings,
        latents=latents,
        loading_bar=False,
        height=height,
        width=width,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        batch_size=batch_size,
    )
    im.save(im_file)


In [ ]:
# Concept + Latent interp

In [ ]:
from typing import Tuple, List
from pydantic import BaseModel


class Config(BaseModel):
    prompt_list: List[str]
    
        
    n_steps_latent: int = 60
    n_steps_prompt: int = 360


class _Step(BaseModel):
    # (Prompt, Weight)
    # [("prompt_1", 0.5), ...]
    prompts: List[Tuple[str, float]]
        
    # (Seed integer, Weight)
    latents: List[Tuple[int, float]]
        
        
def generate_steps(config: Config):
    latent_fracs = np.linspace(0, 1, config.n_steps_latent)
    prompt_fracs = np.linspace(0, 1, config.n_steps_prompt)
    
    c = 0
    steps = []
    
    for prompt_i, prompt in enumerate(config.prompt_list[:-1]):
        prompt_2 = config.prompt_list[prompt_i + 1]
        for prompt_frac in prompt_fracs:
            
            # calc latent seeds
            latent_frac = latent_fracs[c % config.n_steps_latent]
            seed_1 = c // config.n_steps_latent
            seed_2 = seed_1 + 1
            
            steps.append(
                _Step(
                    prompts=[(prompt, 1 - prompt_frac), (prompt_2, prompt_frac)],
                    latents=[(seed_1, 1 - latent_frac), (seed_2, latent_frac)],
                )
            )
            
            c += 1
    return steps


def generate_latent(latent_spec: List[Tuple[int, float]]) -> torch.Tensor:
    latent = sum([
        frac * torch.randn(
            (
                batch_size, 
                unet.in_channels, 
                height // 8, 
                width // 8
            ),
            generator=torch.manual_seed(seed),
        )
        for seed, frac in latent_spec
    ])
    latent -= latent.mean()
    latent /= latent.std()
    return latent
            


def generate_images(
    steps: List[_Step], 
    output_dir: str,
    n_scout_images: int = 20
):
    
    uncond_embeddings, _ = prep_text([""], tokenizer)
    
    all_prompts = list({prompt for step in steps for prompt, _ in step.prompts})
    embeddings = {
        prompt: prep_text([prompt], tokenizer)[0]
        for prompt in list(set(all_prompts))
    }
    
    enumerated_steps = list(enumerate(steps))
    scout_step = len(enumerated_steps) // n_scout_images
    
    print(scout_step)
    print(len(enumerated_steps[::scout_step]))
    print(len([s for s in enumerated_steps if (s[0] % scout_step) != 0]))
    ordered_steps = enumerated_steps[::scout_step] + [s for s in enumerated_steps if (s[0] % scout_step) != 0]
    
    for c, step in tqdm(ordered_steps):
        
        im_file = output_dir / f"image_{c:05d}.png"
        
        if im_file.exists():
            continue

        latent = generate_latent(step.latents)
        
        embedding = sum([
            embeddings[prompt] * frac
            for prompt, frac in step.prompts
        ])
        embedding = torch.cat([uncond_embeddings, embedding])

        im = generate_image(
            embedding,
            latents=latent,
            loading_bar=False,
            height=height,
            width=width,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            batch_size=batch_size,
        )
        im.save(im_file)


In [ ]:
current_output_dir = get_output_dir(Path(output_dir) / "interp_latent_and_embeddings")

height = 512
width = 768
num_inference_steps = 50
guidance_scale = 7.5
generator = torch.manual_seed(42)
batch_size = 1

prompt_list = [
    "An octopus, national geographic, 4k, high resolution",
    "A robotic octopus, steampunk",
    "A messy pile of wires, tangled, mess",
    "A bowl of spaghetti, overflowing",
    "Flying spaghetti monster in the sky",
    "Flying spaghetti monster, ancient drawing, hieroglyphics, worshipers",
]


_steps_factor = 5
n_steps_latent = 60 * _steps_factor
n_steps_prompt = 360 * _steps_factor
    
config = Config(
    prompt_list=prompt_list
)

steps = generate_steps(config)

generate_images(steps, current_output_dir)

In [ ]:
def generate_sin_wave_embeddings(
    prompt_1: str, 
    prompt_2: str, 
    n_periods: int = 5,
    points_per_period: int = 360,
    points_per_seed: int = 180,
    amplitude: float = 0.2,
    mean: float = 0.5,
):
    prompt_fracs = np.sin(np.linspace(
        0, 
        2*np.pi*n_periods, 
        points_per_period*n_periods
    )) * amplitude + mean
    
    latent_fracs = np.linspace(0, 1, points_per_seed)
    
    steps = []
    
    for i, prompt_frac in enumerate(prompt_fracs):

        # calc latent seeds
        latent_frac = latent_fracs[c % config.n_steps_latent]
        seed_1 = i // points_per_seed
        seed_2 = seed_1 + 1
        
        
        steps.append(
            _Step(
                prompts=[(prompt_1, 1 - prompt_frac), (prompt_2, prompt_frac)],
                latents=[(seed_1, 1 - latent_frac), (seed_2, latent_frac)],
            )
        )
        
    return steps
        

In [ ]:
steps = generate_sin_wave_embeddings(
    "A frog catching a fly",
    "A swan swimming in a lake",
)
generate_images(
    steps,
    get_output_dir(Path(output_dir) / "interp_latent_and_embeddings"),
)

In [ ]:
current_output_dir = get_output_dir(Path(output_dir) / "interp_latent_and_embeddings")

steps = generate_sin_wave_embeddings(
    "A swan swimming in a lake",
    "A horse grazing in a field of flowers",
    amplitude=0.1,
    points_per_seed=100,
    n_periods=10,
)

generate_images(
    steps,
    current_output_dir,
)

In [ ]:
def generate_overshoot_steps(
    prompt_1: str,
    prompt_2: str,
    n_points: int = 1000,
    points_per_seed: int = 180,
):
    half_cycle = np.linspace(0, np.pi, n_points // 2)
    frac_list = np.concatenate([np.cos(half_cycle) / 2 + 1, np.cos(half_cycle) / 2])
    latent_fracs = np.linspace(0, 1, points_per_seed)
    
    steps = []
    for i, frac in enumerate(frac_list):
        
        # calc latent seeds
        latent_frac = latent_fracs[c % config.n_steps_latent]
        seed_1 = i // points_per_seed
        seed_2 = seed_1 + 1
        
        steps.append(_Step(
            prompts=[
                (prompt_1, frac),
                (prompt_2, 1 - frac),
            ],
            latents=[
                (seed_1, 1 - latent_frac),
                (seed_2, latent_frac),
            ]
        ))
        
    return steps

In [ ]:
# current_output_dir = get_output_dir(Path(output_dir) / "interp_latent_and_embeddings")

steps = generate_overshoot_steps(
    "A watercolor painting of a bookshelf",
    "A pencil schematic of a complex engine",
    n_points=2000,
    points_per_seed=100
)

generate_images(
    steps,
    current_output_dir,
)

In [ ]:
# current_output_dir = get_output_dir(Path(output_dir) / "interp_latent_and_embeddings")

height = 512
width = 768
num_inference_steps = 50
guidance_scale = 7.5
generator = torch.manual_seed(42)
batch_size = 1

prompt_list = [
    "A giant bird flying above an ocean",
    "A giant steampunk bird, flying above an industrial zone, emitting steam",
    "A giant robotic pterodactyl",
    "A pterodactyl flying above a pre-historic landscape",
    "A flock of pterodactyls standing around some food",
    "A giant meteor entering the atmostphere above a landscape of dinosaurs",
    "A world covered in smoke and ash",
    "The fossil of a pterodactyl, dig site, archaeology",
]


_steps_factor = 5
n_steps_latent = 60 * _steps_factor
n_steps_prompt = 360 * _steps_factor
    
config = Config(
    prompt_list=prompt_list
)

steps = generate_steps(config)

generate_images(steps, current_output_dir)

In [ ]:
current_output_dir = get_output_dir(Path(output_dir) / "interp_latent_and_embeddings")

prompt_list = [
    "Milford sound",
    "Beautiful fjord",
    "Beautiful fjord with a wooden cabin",
    "Clouds rolling over a fjord with a wooden cabin",
    "Rain clouds above a fjord with a wooden cabin",
    "A wooden cabin in a fjord, torrential rain",
    "Looking through the window of a cabin, people around the fireplace, raining outside",
    "a game of cards around the fireplace, wooden cabin",
    "A gentle fire in the fireplace of a wooden cabin",
    "Embers in the fireplace of a wooden cabin"
]


_steps_factor = 7
n_steps_latent = 180 * _steps_factor
n_steps_prompt = 360 * _steps_factor
    
config = Config(
    prompt_list=prompt_list,
    n_steps_latent=n_steps_latent,
    n_steps_prompt=n_steps_prompt,
)

steps = generate_steps(config)

generate_images(
    steps,
    current_output_dir,
    n_scout_images=len(prompt_list)**3,
)

# Scheduling and Visualisation

In [ ]:
im = Image.open('/content/drive/MyDrive/stablediff/shitake3.png').convert('RGB')
im = im.resize((512,512))
encoded = pil_to_latent(im)
im

# Image to Image

In [ ]:
# Make a folder to store results
!rm -rf /content/drive/MyDrive/stablediff/noise2
!mkdir -p /content/drive/MyDrive/stablediff/noise2

# View a noised version
noise = torch.randn_like(encoded) # Random noise

for i in tqdm(range(200)):
    timestep = i * 5
    encoded_and_noised = scheduler.add_noise(encoded, noise, timestep)
    img = latents_to_pil(encoded_and_noised)[0]
    img.save(f'/content/drive/MyDrive/stablediff/noise/{i:04}.jpeg')

!ffmpeg -v 1 -y -f image2 -framerate 12 -i /content/drive/MyDrive/stablediff/noise/%04d.jpeg -c:v libx264 -preset slow -qp 18 -pix_fmt yuv420p out.mp4
mp4 = open('out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=512 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)


In [ ]:
prompt = ["wooden carving of a rabbit eating a leaf, highly detailed, 4k, artisan"]
height = 512                       
width = 512                        
num_inference_steps = 50   
guidance_scale = 8            
generator = torch.manual_seed(1)   
batch_size = 1

# Prep text 
text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0] 
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

# Prep Scheduler
scheduler.set_timesteps(num_inference_steps)

# Start step
start_step = 25
start_sigma = scheduler.sigmas[start_step]
start_timestep = int(scheduler.timesteps[start_step])

# Prep latents
noise = torch.randn_like(encoded)
latents = scheduler.add_noise(encoded, noise, start_timestep)
latents = latents.to(torch_device)
latents = latents * start_sigma  # << NB

# Loop
with autocast("cuda"):
  for i, t in tqdm(enumerate(scheduler.timesteps)):
    if i > start_step:
      # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
      latent_model_input = torch.cat([latents] * 2)
      sigma = scheduler.sigmas[i]
      latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

      # predict the noise residual
      with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]

      # perform guidance
      noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
      noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

      # compute the previous noisy sample x_t -> x_t-1
      latents = scheduler.step(noise_pred, i, latents)["prev_sample"]

latents_to_pil(latents)[0]

# Mixed guidance

In [ ]:
prompts = ['blue fire', 'reticulated python in a tree']
weights = [0.5,0.5]
height = 512                
width = 768                      
num_inference_steps = 50 
guidance_scale = 8 
generator = torch.manual_seed(5)
batch_size = 1

# Prep text 
# Embed both prompts
text_embeddings = []
for i in range(len(prompts)):
    text_input = tokenizer([prompts[i]], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
        text_embeddings.append(text_encoder(text_input.input_ids.to(torch_device))[0])

# Take the average
weighted_embeddings = torch.zeros(text_embeddings[0].shape).to(torch_device)

with torch.no_grad():
    for i in range(len(prompts)):
        weighted_embeddings.add_(text_embeddings[i] * weights[i])

text_embeddings = weighted_embeddings

# And the uncond. input as before:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0] 
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

# Prep Scheduler
scheduler.set_timesteps(num_inference_steps)

# Prep latents
latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(torch_device)
latents = latents * scheduler.sigmas[0] # Need to scale to match k

# Loop
with autocast("cuda"):
  for i, t in tqdm(enumerate(scheduler.timesteps)):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    latent_model_input = torch.cat([latents] * 2)
    sigma = scheduler.sigmas[i]
    latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

    # predict the noise residual
    with torch.no_grad():
      noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

    # compute the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, i, latents)["prev_sample"]

latents_to_pil(latents)[0]

# Denoising Example

In [ ]:
prompt = 'a futuristic city, abandoned and overgrown with plants, dystopia, bathed in sunlight'
height = 512                     
width = 768 
num_inference_steps = 100 
guidance_scale = 8     
generator = torch.manual_seed(10) 
batch_size = 1

# Make a folder to store results
!rm -rf /content/drive/MyDrive/stablediff/denoising2/
!mkdir -p /content/drive/MyDrive/stablediff/denoising2/

# Prep text 
text_input = tokenizer([prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
with torch.no_grad():
  text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]

# And the uncond. input as before:
max_length = text_input.input_ids.shape[-1]
uncond_input = tokenizer(
    [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
)
with torch.no_grad():
  uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0] 
text_embeddings = torch.cat([uncond_embeddings, text_embeddings])

# Prep Scheduler
scheduler.set_timesteps(num_inference_steps)

# Prep latents
latents = torch.randn(
  (batch_size, unet.in_channels, height // 8, width // 8),
  generator=generator,
)
latents = latents.to(torch_device)
latents = latents * scheduler.sigmas[0] # Need to scale to match k

# Loop
with autocast("cuda"):
  for i, t in tqdm(enumerate(scheduler.timesteps)):
    # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
    
    im_input = latents_to_pil(latents)[0]

    latent_model_input = torch.cat([latents] * 2)
    sigma = scheduler.sigmas[i]
    latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5)

    # predict the noise residual
    with torch.no_grad():
      noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings)["sample"]

    # perform guidance
    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
    
    # Get the predicted x0:
    latents_x0 = latents - sigma * noise_pred

    im_t0 = latents_to_pil(latents_x0)[0]
    im_noise = latents_to_pil(sigma * noise_pred)[0]

    # And the previous noisy sample x_t -> x_t-1
    latents = scheduler.step(noise_pred, i, latents)["prev_sample"]
    im_next = latents_to_pil(latents)[0]

    # Combine the two images and save for later viewing
    im = Image.new('RGB', (2304, 512))
    im.paste(im_input, (0, 0))
    im.paste(im_noise, (768, 0))
    im.paste(im_t0, (1536, 0))
    im.save(f'/content/drive/MyDrive/stablediff/denoising2/{i:04}.jpg')

In [ ]:
!ffmpeg -v 1 -y -f image2 -framerate 12 -i /content/drive/MyDrive/stablediff/denoising2/%04d.jpg -c:v libx264 -preset slow -qp 18 -pix_fmt yuv420p out.mp4
mp4 = open('out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1536 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)